In [31]:
import os
import numpy as np
import pandas as pd
from IPython.display import Image
import cv2
from glob import glob
import re
import yaml

In [32]:
yolo_base = './yolov11'
sn_track_base = './data/SoccerNetGS'

# SoccerNet data setting

In [33]:
# Of the 57 half , 50 are used for training and 7 for validation. The class of other is in SNGS-166 and SNGS-076 (one for train and one for validation).
trn_dirs = sorted(glob(f'{sn_track_base}/train/SNGS*'))
#tst_dirs = sorted(glob(f'{sn_track_base}/test/SNGS*'))
tst_dirs = trn_dirs[50:]
trn_dirs = trn_dirs[:50]

debug=False
if debug:
    trn_dirs = trn_dirs[:5]
    tst_dirs = tst_dirs[:5]

split_dirs = {
    'train':trn_dirs,
    'valid':tst_dirs
}

# 5 types of labels are used.
labels = ['player', 'goalkeeper', 'referee', 'ball', 'other']
# label_dict = {'player': 0, 'goalkeeper': 1, 'referee': 2, 'ball': 3, 'other': 4}


# yolov11 setting
Only three types of files need to be prepared:  
(1) image files  
(2) label files  
(3) configuration files

In [34]:
#(1) image file path
yolo_train_img_dir = f'{yolo_base}/images/train'
yolo_valid_img_dir = f'{yolo_base}/images/valid'

#(2) label file path
yolo_train_label_dir = f'{yolo_base}/labels/train'
yolo_valid_label_dir = f'{yolo_base}/labels/valid'

#(3) config file path
yaml_file = f'{yolo_base}/data.yaml'

os.makedirs(yolo_train_img_dir, exist_ok=True)
os.makedirs(yolo_valid_img_dir, exist_ok=True)
os.makedirs(yolo_train_label_dir, exist_ok=True)
os.makedirs(yolo_valid_label_dir, exist_ok=True)

In [35]:
import json 

# convert from x,y,w,h to yolo format
def get_yolo_format_bbox(img_w, img_h, box):
    w = box["w"]
    h = box["h"]
    xc = box["x_center"]
    yc = box["y_center"]
    box = [xc/img_w, yc/img_h, w/img_w, h/img_h]
    box = [f"{i:.4g}" for i in box]
    return box

for split in split_dirs:
    print('split:',split)
    if split == 'train':
        yolo_img_dir = yolo_train_img_dir
        yolo_label_dir = yolo_train_label_dir
    else:
        yolo_img_dir = yolo_valid_img_dir
        yolo_label_dir = yolo_valid_label_dir

    for this_dir in split_dirs[split]:
        print('this_dir:',this_dir)

        # open the Json file
        with open(f'{this_dir}/Labels-GameState.json') as f:
            data = json.load(f)
        

        img_dir_of_batch = data["info"]["im_dir"]
        batch_name = data["info"]["name"]


        # For each image id in image, take the image_id, file_name, height and width and
        # all the annotations corresponding to that image_id
        for image in data['images']:
            # get the image id
            image_id = image['image_id']
            # get the file name
            file_name = image['file_name'].split('.')[0]
            # get the height and width
            height = image['height']
            width = image['width']
            new_file_name = f"{batch_name}_{file_name}"

            # get the annotations corresponding to that image id
            annotations = [x for x in data['annotations'] if x['image_id'] == image_id]

            # create a list of boxes
            boxes = []
            for annotation in annotations:
                category_id = annotation['category_id']
                # Skip the pitch category becaus it does not have a bbox
                if category_id == 5:
                    continue
                # Remap tha label to match the yolo format
                category_id -= 1

                # To map the other category, TODO: this shoudl be done with remap of a dict
                if category_id == 6:
                    category_id -= 2
                
                bbox = annotation['bbox_image']
                normalized_bbox = get_yolo_format_bbox(width, height, bbox)
                box = []
                box = [category_id] + normalized_bbox
                boxes.append(box)

            # write the boxes to a file
            if len(boxes) > 0:
                with open(f'{yolo_label_dir}/{new_file_name}.txt', 'w') as f:
                    for box in boxes:
                        f.write(' '.join(map(str, box)) + '\n')
            
            # copy the image to the yolo directory
            os.system(f'cp {this_dir}/{img_dir_of_batch}/{file_name}.jpg {yolo_img_dir}/{new_file_name}.jpg')

        

split: train
this_dir: ./data/SoccerNetGS/train/SNGS-060
this_dir: ./data/SoccerNetGS/train/SNGS-061
this_dir: ./data/SoccerNetGS/train/SNGS-062
this_dir: ./data/SoccerNetGS/train/SNGS-063
this_dir: ./data/SoccerNetGS/train/SNGS-064
this_dir: ./data/SoccerNetGS/train/SNGS-065
this_dir: ./data/SoccerNetGS/train/SNGS-066
this_dir: ./data/SoccerNetGS/train/SNGS-067
this_dir: ./data/SoccerNetGS/train/SNGS-068
this_dir: ./data/SoccerNetGS/train/SNGS-069
this_dir: ./data/SoccerNetGS/train/SNGS-070
this_dir: ./data/SoccerNetGS/train/SNGS-071
this_dir: ./data/SoccerNetGS/train/SNGS-072
this_dir: ./data/SoccerNetGS/train/SNGS-073
this_dir: ./data/SoccerNetGS/train/SNGS-074
this_dir: ./data/SoccerNetGS/train/SNGS-075
this_dir: ./data/SoccerNetGS/train/SNGS-076
this_dir: ./data/SoccerNetGS/train/SNGS-077
this_dir: ./data/SoccerNetGS/train/SNGS-097
this_dir: ./data/SoccerNetGS/train/SNGS-098
this_dir: ./data/SoccerNetGS/train/SNGS-099
this_dir: ./data/SoccerNetGS/train/SNGS-100
this_dir: ./data/So

In [36]:
abs_yolo_train_img_dir = os.path.abspath(yolo_train_img_dir)
abs_yolo_valid_img_dir = os.path.abspath(yolo_valid_img_dir)

# Dump config file
data_yaml = dict(
    train = abs_yolo_train_img_dir,
    val = abs_yolo_valid_img_dir,
    nc = 5,
    names = labels
)

with open(yaml_file, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

# Install YOLO

In [39]:
# !pip install ultralytics --upgrade

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.143 🚀 Python-3.9.22 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
Setup complete ✅ (8 CPUs, 62.7 GB RAM, 577.8/915.3 GB disk)


In [ ]:
from ultralytics import YOLO
import torch
import gc

# free torch GPU memory
torch.cuda.empty_cache()
# free GPU memory
gc.collect()

model = YOLO('/home/student/headers-tracking/my_gamestate/pretrained_models/yolo/yolo11m.pt')
data = os.path.abspath("./yolov11/data.yaml")
# epochs = 25
epochs = 100  # For full training
dropout = 0.0  
# dropout = 0.1  # Use dropout if overfitting, otherwise keep it 0
batch = 8 


model.train(
    data=data,
    model=model,
    epochs=epochs,  
    imgsz=1024,              # Higher resolution to improve detection on small objects
    batch=batch,              # 16 should fit on a 2080 Ti at 1280px; if OOM, reduce to 8 or try 1024 imgsz
    lr0=0.01,                # Initial learning rate (default good starting point)
    lrf=0.01,                # Learning rate factor (final_lr = lr0 * lrf)
    weight_decay=0.0005,     # L2 Regularization
    momentum=0.937,          # SGD momentum
    warmup_epochs=3,         # Gradual ramp-up
    rect=True,               # Use aspect-ratio padding for varied input shapes
    workers=8,               # CPU data loading workers
    device=0,                # CUDA:0 = use your GPU
    optimizer='SGD',         # SGD typically works best for YOLO models
    patience=20,             # For early stopping, if validation stops improving
    dropout=dropout,             # Use only if overfitting; otherwise keep 0
    val=True,                # Enable validation after every epoch
    save=True,               # Save weights
    plots=True,              # Save training plots
    name='finetune_yolov11m' # Output folder
)



New https://pypi.org/project/ultralytics/8.3.144 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.143 🚀 Python-3.9.22 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/student/headers-tracking/my_gamestate/yolov11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=YOLO(
  (

train: Scanning /home/student/headers-tracking/my_gamestate/yolov11/labels/train.cache... 37500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37500/37500 [00:00<?, ?it/s]


albumentations: __init__() got an unexpected keyword argument 'quality_range'
WARNING ⚠️ 'rect=True' is incompatible with DataLoader shuffle, setting shuffle=False
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1449.1±1092.4 MB/s, size: 243.2 KB)


val: Scanning /home/student/headers-tracking/my_gamestate/yolov11/labels/valid.cache... 5250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5250/5250 [00:00<?, ?it/s]


Plotting labels to runs/detect/finetune_yolov11m4/labels.jpg... 
optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs/detect/finetune_yolov11m4
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       5250      89818      0.898      0.677      0.709      0.442
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 16, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

36 epochs completed in 9.067 hours.
Optimizer stripped from runs/detect/finetune_yolov11m4/weights/last.pt, 40.6MB
Optimizer stripped from runs/detect/finetune_yolov11m4/weights/best.pt, 40.6MB

Validating runs/detect/finetune_yolov11m4/weights/best.pt...
Ultralytics 8.3.143 🚀 Python-3.9.22 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
YOLO11m summary (fused): 125 layers, 20,033,887 parameters, 0 gradients, 67.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 329/329 [00:52<00:00,  6.23it/s]


                   all       5250      89818      0.892      0.667      0.734       0.44
                player       5250      73311      0.946      0.973      0.974      0.667
            goalkeeper       2870       2870      0.914      0.957      0.963      0.623
               referee       4872       7974      0.914      0.957      0.969      0.639
                  ball       4992       5400      0.688       0.45      0.453       0.17
                 other        263        263          1          0       0.31      0.101
Speed: 0.2ms preprocess, 6.5ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/finetune_yolov11m4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x732bd73f1940>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

: 

In [30]:
!yolo task=detect mode=predict model=/home/student/headers-tracking/my_gamestate/pretrained_models/yolo/yolo11x.pt source=/home/student/headers-tracking/my_gamestate/yolov11/images/valid/SNGS-167_000220.jpg save=True
!yolo task=detect mode=predict model=/home/student/headers-tracking/my_gamestate/pretrained_models/yolo/yolo11m.pt source=/home/student/headers-tracking/my_gamestate/yolov11/images/valid/SNGS-167_000220.jpg save=True
!yolo task=detect mode=predict model=/home/student/headers-tracking/my_gamestate/pretrained_models/yolo/yolo11n.pt source=/home/student/headers-tracking/my_gamestate/yolov11/images/valid/SNGS-167_000220.jpg save=True
!yolo task=detect mode=predict model=/home/student/headers-tracking/my_gamestate/runs/detect/finetune_yolov11x3/weights/best.pt source=/home/student/headers-tracking/my_gamestate/yolov11/images/valid/SNGS-167_000220.jpg save=True

Ultralytics 8.3.143 🚀 Python-3.9.22 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
YOLO11x summary (fused): 190 layers, 56,919,424 parameters, 0 gradients, 194.9 GFLOPs

image 1/1 /home/student/headers-tracking/my_gamestate/yolov11/images/valid/SNGS-167_000220.jpg: 384x640 14 persons, 23.5ms
Speed: 2.3ms preprocess, 23.5ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict
Ultralytics 8.3.143 🚀 Python-3.9.22 torch-1.13.1 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
YOLO11m summary (fused): 125 layers, 20,091,712 parameters, 0 gradients, 68.0 GFLOPs

image 1/1 /home/student/headers-tracking/my_gamestate/yolov11/images/valid/SNGS-167_000220.jpg: 384x640 15 persons, 14.9ms
Speed: 2.3ms preprocess, 14.9ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict
Ultr